# Comparison against First Best

The goal here is to compare both the traditional and mechanism-implemented game against the first best option.

In [1]:
import os
import glob
import numpy as np
import pandas as pd
import math
import random
import itertools
from typing import List, Dict
import matplotlib.pyplot as plt
import seaborn as sns
from collections import namedtuple, defaultdict
from pandas.plotting import parallel_coordinates
# from PSSimPy.simulator import ABMSim
from PSSimPy import Bank, Account
from PSSimPy.credit_facilities import AbstractCreditFacility
from PSSimPy.utils import add_minutes_to_time, minutes_between

from MDP import MechMDPSearch, MDPStateExt, TradMDPSearch, MDPStateTrad
from SimClasses import ABMSim

## Setup